# ***LOAD: orders.csv***


In [ ]:
import pandas as pd
orders=pd.read_csv("orders.csv")
orders.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name
0,1,2508,450,18-02-2023,842.97,New Foods Chinese
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian


# ***LOAD: users.json***

In [ ]:
import pandas as pd
users = pd.read_json("users.json")
users.head()

,user_id,name,city,membership
0,1,User_1,Chennai,Regular
1,2,User_2,Pune,Gold
2,3,User_3,Bangalore,Gold
3,4,User_4,Bangalore,Regular
4,5,User_5,Pune,Gold


# ***LOAD: restaurants.sql***

In [ ]:
import sqlite3
import pandas as pd


creating the DB and load the file:

In [ ]:
conn = sqlite3.connect("restaurants.db")
with open("restaurants.sql", "r", encoding="utf-8") as file:
    sql_data = file.read()
    file.close()
conn.executescript(sql_data)


Read restaurant table:

In [ ]:
import pandas as pd
restaurants = pd.read_sql_query("SELECT * FROM restaurants", conn)
restaurants.head()


,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


# **MERGE Orders and Users:**

In [ ]:
orders_users = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)
orders_users.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name,name,city,membership
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular


MERGE Restaurants with above table

In [ ]:
final_df = pd.merge(
    orders_users,
    restaurants,
    on="restaurant_id",
    how="left"
)
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


**SAVE the Final Dataset:**

In [ ]:
final_df.to_csv("final_food_delivery_dataset.csv",index=False)

# ***ANSWERS:***

In [ ]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

Q1. Which city has the highest total revenue (total_amount) from Gold members?  

In [ ]:
gold_df = df[df["membership"] == "Gold"]

city_revenue = gold_df.groupby("city")["total_amount"].sum()

city_revenue = city_revenue.sort_values(ascending=False)

city_revenue


,total_amount
city,
Chennai,1080909.79
Pune,1003012.32
Bangalore,994702.59
Hyderabad,896740.19


Q2. Which cuisine has the highest average order value across all orders?

In [ ]:
cuisine_group = df.groupby("cuisine")
avg_order_value = cuisine_group["total_amount"].mean()
avg_order_value = avg_order_value.sort_values(ascending=False)
avg_order_value


,total_amount
cuisine,
Mexican,808.021344
Italian,799.448578
Indian,798.466011
Chinese,798.389020


Q3. How many distinct users placed orders worth more than ₹1000 in total (sum of all their orders)?

In [ ]:
user_group = df.groupby("user_id")
user_total = user_group["total_amount"].sum()
high_value_users = user_total[user_total > 1000]
high_value_users.count()


np.int64(2544)

Q4. Which restaurant rating range generated the highest total revenue?




In [ ]:
df["rating_range"] = pd.cut(
    df["rating"],
    bins=[0, 2, 3, 4, 5],
    labels=["0-2", "2-3", "3-4", "4-5"]
)

#group by rating range
rating_group = df.groupby("rating_range")
rating_revenue = rating_group["total_amount"].sum()

#sort
rating_revenue = rating_revenue.sort_values(ascending=False)
rating_revenue


/tmp/ipython-input-3408063995.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  rating_group = df.groupby("rating_range")


,total_amount
rating_range,
4-5,4157357.01
3-4,3599248.98
2-3,255018.13
0-2,0.00


Q5. Among Gold members, which city has the highest average order value?


In [ ]:
gold_df = df[df["membership"] == "Gold"]

avg_order_value = gold_df.groupby("city")["total_amount"].mean()

avg_order_value = avg_order_value.sort_values(ascending=False)
avg_order_value


,total_amount
city,
Chennai,808.459080
Hyderabad,806.421034
Bangalore,793.223756
Pune,781.162243


Q6. Which cuisine has the lowest number of distinct restaurants but still contributes significant revenue?


In [ ]:
cuisine_summary = cuisine_summary.sort_values(
    by=["restaurant_count", "total_revenue"],
    ascending=[True, False]
)

cuisine_summary


,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65
Italian,126,2024203.80
Indian,126,1971412.58
Mexican,128,2085503.09


Q7. What percentage of total orders were placed by Gold members? (Rounded to nearest integer)

In [ ]:
total_orders = df["order_id"].count()
gold_orders = df[df["membership"] == "Gold"]["order_id"].count()
percentage = (gold_orders / total_orders) * 100
round(percentage)


50

Q8. Which restaurant has the highest average order value but less than 20 total orders?

In [ ]:
df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'total_amount',
       'restaurant_name_x', 'name', 'city', 'membership', 'restaurant_name_y',
       'cuisine', 'rating', 'rating_range'],
      dtype='object')

In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

df.head()


,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

# group by restaurant name
grouped = df.groupby("restaurant_name_x")

order_count = grouped["order_id"].count()
avg_value = grouped["total_amount"].mean()

summary = pd.DataFrame({
    "order_count": order_count,
    "avg_order_value": avg_value
})

# filter restaurants with less than 20 orders
summary = summary[summary["order_count"] < 20]

# sort by highest average order value
summary = summary.sort_values("avg_order_value", ascending=False)

summary.head()


,order_count,avg_order_value
restaurant_name_x,,
Hotel Dhaba Multicuisine,13,1040.222308
Sri Mess Punjabi,12,1029.180833
Ruchi Biryani Punjabi,16,1002.140625
Sri Delights Pure Veg,18,989.467222
Classic Kitchen Family Restaurant,19,973.167895


In [ ]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")
options = [
    "Grand Cafe Punjabi",
    "Grand Restaurant South Indian",
    "Ruchi Mess Multicuisine",
    "Ruchi Foods Chinese"
]
filtered = df[df["restaurant_name_x"].isin(options)]

stats = filtered.groupby("restaurant_name_x").agg(
    order_count=("order_id", "count"),
    avg_order_value=("total_amount", "mean")
)
stats


,order_count,avg_order_value
restaurant_name_x,,
Grand Cafe Punjabi,32,765.409063
Grand Restaurant South Indian,29,842.567586
Ruchi Foods Chinese,19,686.603158
Ruchi Mess Multicuisine,40,851.226250


Q8. Which combination contributes the highest revenue?


In [ ]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

options = [
    ("Gold", "Indian"),
    ("Gold", "Italian"),
    ("Regular", "Indian"),
    ("Regular", "Chinese")
]
results = []
for mem, cui in options:
    revenue = df[
        (df["membership"] == mem) &
        (df["cuisine"] == cui)
    ]["total_amount"].sum()

    results.append([mem, cui, revenue])

result_df = pd.DataFrame(
    results,
    columns=["membership", "cuisine", "total_revenue"]
)
result_df


,membership,cuisine,total_revenue
0,Gold,Indian,979312.31
1,Gold,Italian,1005779.05
2,Regular,Indian,992100.27
3,Regular,Chinese,952790.91


Q9. During which quarter of the year is the total revenue highest?


In [ ]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

df["order_date"] = pd.to_datetime(df["order_date"])

df["quarter"] = df["order_date"].dt.to_period("Q")

quarter_revenue = df.groupby("quarter")["total_amount"].sum()

quarter_revenue.sort_values(ascending=False)


/tmp/ipython-input-3360023589.py:4: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df["order_date"] = pd.to_datetime(df["order_date"])


,total_amount
quarter,
2023Q3,2037385.10
2023Q4,2018263.66
2023Q1,1993425.14
2023Q2,1945348.72
2024Q1,17201.50


How many total orders were placed by users with Gold membership?


In [15]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

gold_orders = df[df["membership"] == "Gold"]["order_id"].count()
gold_orders


np.int64(4987)

What is the total revenue (rounded to nearest integer) generated from orders placed in Hyderabad city?


In [17]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

hyd_revenue = df[df["city"] == "Hyderabad"]["total_amount"].sum()
round(hyd_revenue)


1889367

How many distinct users placed at least one order?


In [19]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

df["user_id"].nunique()


2883

What is the average order value (rounded to 2 decimals) for Gold members?


In [21]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

round(df[df["membership"]=="Gold"]["total_amount"].mean(),2)

np.float64(797.15)

How many orders were placed for restaurants with rating ≥ 4.5?


In [22]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

df[df["rating"] >= 4.5]["order_id"].count()

np.int64(3374)

How many orders were placed in the top revenue city among Gold members only?


In [23]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

#filtering  gold members
gold_df = df[df["membership"] == "Gold"]

#finding top revenue city among gold members
city_revenue = gold_df.groupby("city")["total_amount"].sum()
top_city = city_revenue.idxmax()

#counting orders in that city among gold members
gold_df[gold_df["city"] == top_city]["order_id"].count()


np.int64(1337)

Total no of rows in final csv file

In [24]:
import pandas as pd
df = pd.read_csv("final_food_delivery_dataset.csv")

len(df)


10000